## 1. Pretraining 전처리

라벨 : 없음

In [1]:
import os
import numpy as np
from scipy.io import loadmat

In [2]:
data_dir = "../../Database/raw/Healthy_Subjects"

# x3은 너무 데이터가 flat해서 버렸음음
selected_channels = ['F3_A2', 'C3_A2', 'O1_A2', 'F4_A1', 'C4_A1',
                     'O2_A1', 'LOC_A2', 'ROC_A1', 'X1', 'X2']

### 데이터 정규화

전체를 같은 정규화x

채널별 Z-score 정규화 방식

In [3]:
X_all = []

# 데이터 로딩 및 정규화
for fname in sorted(os.listdir(data_dir)):
    if not fname.endswith(".mat"):
        continue

    file_path = os.path.join(data_dir, fname)
    mat_data = loadmat(file_path)

    signals = []
    for ch in selected_channels:
        if ch not in mat_data:
            print(f"⚠️ 채널 {ch} 누락: {fname}")
            continue
        signals.append(mat_data[ch])  # (epochs, time)

    data = np.stack(signals, axis=1)  # (epochs, channels, time)

    # Z-score 정규화
    for c in range(data.shape[1]):
        ch_data = data[:, c, :]
        mean = np.mean(ch_data)
        std = np.std(ch_data) + 1e-6
        data[:, c, :] = (ch_data - mean) / std

    X_all.append(data)

### 병합, 저장

In [4]:
X_all = np.concatenate(X_all, axis=0)  # (전체 epoch, channels, time)
print("전처리 완료!")
print("X shape:", X_all.shape)         # (85890, 10, 6000) 이어야 정상

# 저장
np.save("../../Database/processed/sub/subgroup3_pretrain_X.npy", X_all.astype(np.float32))

전처리 완료!
X shape: (8589, 10, 6000)


In [5]:
print(f"{fname} → {data.shape}")


subject9.mat → (939, 10, 6000)
